__177312 - Tales lelo da Aparecida__

Leia o arquivo *abalone* do exercício 2.  
Faça o preprocessamento do atributo categórico e do atributo de saída como no exercício 2 estandardize todos os atributos numéricos.

In [1]:
import pandas as pd
from sklearn.preprocessing         import scale
from sklearn.linear_model          import LogisticRegression
from sklearn.model_selection       import StratifiedKFold, GridSearchCV
from sklearn.svm                   import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Carrega um DataFrame com os dados do arquivo
df = pd.io.parsers.read_csv('abalone.csv', ',', header=None )

# Converte a 1ª coluna para um dado numérico
df = pd.get_dummies(df, columns=[0])

# Cria um DataFrame com a versão categórica (1:x>13, 0:x<=13) da coluna 8
greaterThan13y = df.apply(lambda row: int(row[8]>13), 1)
# Remove-a
del df[8]

# Estandardiza os dados
x = scale(df.values)
y = greaterThan13y.values

# Separando dados para treino e teste
x_train, y_train = x[:3133], y[:3133]
x_test, y_test = x[3133:], y[3133:]

# Renomeia as colunas para facilitar o acompanhamento
df = pd.DataFrame(data=x[:5], columns=['1', '2', '3', '4', '5', '6', '7', 'F', 'I', 'M'])
df['>13y'] = greaterThan13y

# Mostra as primeiras linhas de nossos dados, para um preview de como estão até então
pd.options.display.float_format = '{:,.3f}'.format
print("Primeiras 5 linhas:\n", df[:5])

Primeiras 5 linhas:
        1      2      3      4      5      6      7      F      I      M  >13y
0 -0.575 -0.432 -1.064 -0.642 -0.608 -0.726 -0.638 -0.675 -0.688  1.317     1
1 -1.449 -1.440 -1.184 -1.230 -1.171 -1.205 -1.213 -0.675 -0.688  1.317     0
2  0.050  0.122 -0.108 -0.309 -0.463 -0.357 -0.207  1.482 -0.688 -0.759     0
3 -0.699 -0.432 -0.347 -0.638 -0.648 -0.608 -0.602 -0.675 -0.688  1.317     0
4 -1.616 -1.541 -1.423 -1.272 -1.216 -1.287 -1.321 -0.675  1.453 -0.759     0


# Logistic regression
Faça o logistic regression com $C=10^{-1,0,1,2,3}$. O loop externo deve ser um 5-fold CV estratificado. O loop interno para a escolha do hiperparâmetro deve ser um 3-fold estratificado.
Voce tem que fazer o loop interno explicitamente, usando StratifiedKFold e não funções como GridSearchCV
qual a acurácia do LR com a melhore escolha de parâmetros (para cada fold)?

In [2]:
# Retorna duas listas com os índices gerados pelo kfold estratificado
def getStratifiedKFoldArrays(k, x, y):
    indexes_pairs = list(StratifiedKFold(k).split(x, y))
    train_indexes, test_indexes = [], []
    for i in range(k):
        train_indexes.append(indexes_pairs[i][0])
        test_indexes.append(indexes_pairs[i][1])
    return train_indexes, test_indexes


# Possíveis hiperparâmetros
Cs = [1e-1, 1, 1e+1, 1e+2, 1e+3]

# Gera os índices dos 5 folds (kfold estratificado)
tr_5kf_idxs, ts_5kf_idxs = getStratifiedKFoldArrays(5, x_train, y_train)

avg_log_score = 0
for tr_5kf_idx, ts_5kf_idx in zip(tr_5kf_idxs, ts_5kf_idxs):
    # Define os subvetores x e y, para treino e teste (5-kfold)
    x_5kf_tr, y_5kf_tr = x_train[tr_5kf_idx], y_train[tr_5kf_idx] # Treino
    x_5kf_ts, y_5kf_ts = x_train[ts_5kf_idx], y_train[ts_5kf_idx] # Teste

    # Gera os indices dos 3 folds (kfold estratificado)
    # Vale destacar que estes índices são relativos a x_train e y_train
    tr_3kf_idxs, ts_3kf_idxs = getStratifiedKFoldArrays(3, x_5kf_tr, y_5kf_tr)
    
    # Loop verificador do melhor hiperparâmetro para o fold atual (5-kfold)
    bestC, bestC_score = Cs[0], 0
    for C in Cs:
        avg_C_score = 0
        for tr_3kf_idx, ts_3kf_idx in zip(tr_3kf_idxs, ts_3kf_idxs):
            # Define os subvetores x e y, para treino e teste (3-kfold)
            x_3kf_tr, y_3kf_tr = x_train[tr_3kf_idx], y_train[tr_3kf_idx] # Treino
            x_3kf_ts, y_3kf_ts = x_train[ts_3kf_idx], y_train[ts_3kf_idx] # Teste
            
            # Treina e calcula a acurácia de C para o fold atual (3-kfold)
            avg_C_score += LogisticRegression(C=C).fit(x_3kf_tr, y_3kf_tr).score(x_3kf_ts, y_3kf_ts)
            
        # Calcula a média da acurácia dos 3 folds
        avg_C_score /= 3
        
        if (avg_C_score > bestC_score):
            bestC, bestC_score = C, avg_C_score
            
    # Guarda a acurácia do LogisticRegression para o melhor C do fold atual
    log_score = LogisticRegression(C=bestC).fit(x_5kf_tr, y_5kf_tr).score(x_5kf_ts, y_5kf_ts)
    print('Acurácia para C = {:5.1f}: {:.12f}'.format(bestC, log_score))
    avg_log_score += log_score
    
avg_log_score /= 5
print("Acurácia do LogisticRegression:", avg_log_score)
    

Acurácia para C =  10.0: 0.885167464115
Acurácia para C = 100.0: 0.904306220096
Acurácia para C =  10.0: 0.891547049442
Acurácia para C =   1.0: 0.899521531100
Acurácia para C =   1.0: 0.897600000000
Acurácia do LogisticRegression: 0.895628452951


# Linear SVM
Faça o LinearSVM com $C=10^{-1,0,1,2,3}$. O loop externo deve ser um 5-fold estratificado. O loop interno um 3-fold estratificado. Neste caso voce não precisa fazer o 3 fold explicitamente, voce pode usar o GridSearchCV. Qual a acurácia do LinearSVM com a melhor escolha de C?

In [3]:
avg_svc_score = 0
for tr_5kf_idx, ts_5kf_idx in zip(tr_5kf_idxs, ts_5kf_idxs):
    # Define os subvetores x e y, para treino e teste (5-kfold)
    x_5kf_tr, y_5kf_tr = x_train[tr_5kf_idx], y_train[tr_5kf_idx] # Treino
    x_5kf_ts, y_5kf_ts = x_train[ts_5kf_idx], y_train[ts_5kf_idx] # Teste
    
    # Procura o melhor C com 3 folds (StratifiedKFold)
    gscv = GridSearchCV(LinearSVC(), {'C':Cs}).fit(x_5kf_tr, y_5kf_tr)
    
    # Guarda a acurácia do LinearSVC para o melhor C do fold atual
    svc_score = gscv.score(x_5kf_ts, y_5kf_ts)
    print('Acurácia para C = {:4.1f}: {:.12f}'.format(gscv.best_params_["C"], svc_score))
    avg_svc_score += svc_score
    
avg_svc_score /= 5
print("Acurácia do LinearSVM :", avg_svc_score)

Acurácia para C = 10.0: 0.888357256778
Acurácia para C =  0.1: 0.894736842105
Acurácia para C =  1.0: 0.889952153110
Acurácia para C =  0.1: 0.904306220096
Acurácia para C =  0.1: 0.892800000000
Acurácia do LinearSVM : 0.894030494418


# LDA
Faça o LDA. Reporte a acurácia.

In [4]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

lda_score = LDA().fit(x_train, y_train).score(x_test, y_test)
print("Acurácia do LDA:", lda_score)

Acurácia do LDA: 0.894636015326


# Classificador final
Qual o melhor classificador para esse problema?
Se não o LDA, calcule o hiperparâmetro C a ser usado. Gere o classificador final.

LR=0.895628452951 > LDA=0.894636015326 > SVM=0.894030494418  
Ou seja, o **LogisticRegression** foi o melhor classificador

In [5]:
# Gera os indices dos 3 folds (kfold estratificado)
tr_3kf_idxs, ts_3kf_idxs = getStratifiedKFoldArrays(3, x, y)

# Loop verificador do melhor hiperparâmetro para o dados de treino
bestC, bestC_score = Cs[0], 0
for C in Cs:
    avg_C_score = 0
    for tr_3kf_idx, ts_3kf_idx in zip(tr_3kf_idxs, ts_3kf_idxs):
        # Define os subvetores x e y, para treino e teste (3-kfold)
        x_3kf_tr, y_3kf_tr = x[tr_3kf_idx], y[tr_3kf_idx] # Treino
        x_3kf_ts, y_3kf_ts = x[ts_3kf_idx], y[ts_3kf_idx] # Teste

        # Treina e calcula a acurácia de C para o fold atual (3-kfold)
        avg_C_score += LogisticRegression(C=C).fit(x_3kf_tr, y_3kf_tr).score(x_3kf_ts, y_3kf_ts)

    # Calcula a média da acurácia dos 3 folds
    avg_C_score /= 3

    if (avg_C_score > bestC_score):
        bestC, bestC_score = C, avg_C_score

print(LogisticRegression(C=bestC).fit(x, y))

LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


Este seria o classificador final, exibido sem pontuação, já que sabemos que o já escolhemos tanto o melhor algoritmo quanto seu melhor hiperparâmetro, podendo, assim, usar todos nossos dados para treino.